In [1]:
P =[1 1 0 0 0 0 0 0 0 0;
    0 0 1 1 0 0 0 0 0 0;
    0 0 0 0 1 1 0 0 0 0;
    0 0 0 0 0 0 1 1 0 0;
    0 0 0 0 0 0 0 0 1 1];

S =[1 1 0 1 0 0 0 0 0 0;
    0 0 1 0 1 0 0 1 0 0;
    0 0 1 0 1 0 0 1 0 0;
    0 0 1 0 1 0 0 1 0 0;
    0 0 1 0 1 0 0 1 0 0;
    1 1 0 1 0 0 0 0 0 0;
    0 0 0 0 0 1 1 0 0 1;
    0 0 0 0 0 1 1 0 0 1;
    0 0 0 0 0 1 1 0 0 1;
    1 1 0 1 0 0 0 0 0 0;
    1 1 0 1 0 0 0 0 0 0;
    0 0 0 0 0 0 1 0 1 1;
    0 0 0 0 0 0 1 0 1 1;
    0 0 0 0 0 0 1 0 1 1;
    0 0 0 0 0 0 1 0 1 1];

nc = 10; # Number of courses
ns = 15; # Number of students
np = 5;  # Number of professors
nt = 6;  # Number of time slots
nr = 4;  # Number of classrooms

print("\nWe will use \"findall\" method to evaluate 4th and 5th constraint effectively")


We will use "findall" method to evaluate 4th and 5th constraint effectively

In [2]:
for p in 1:np
    println(findall(x->x==1, P[p,:]))
end
println("----------")
for s in 1:ns
    println(findall(x->x==1, S[s,:]))
end

[1, 2]
[3, 4]
[5, 6]
[7, 8]
[9, 10]
----------
[1, 2, 4]
[3, 5, 8]
[3, 5, 8]
[3, 5, 8]
[3, 5, 8]
[1, 2, 4]
[6, 7, 10]
[6, 7, 10]
[6, 7, 10]
[1, 2, 4]
[1, 2, 4]
[7, 9, 10]
[7, 9, 10]
[7, 9, 10]
[7, 9, 10]


In [3]:
using JuMP, Cbc
m = Model(with_optimizer(Cbc.Optimizer))
@variable(m, 0<=TimeTables[ i= 1:nc, j = 1:nt, k = 1:nr]<=1 , Bin) #i(course), j(time), k(room)

#CONSTRAINTS
#1.To same place and time, we can schedule at most one course (Same Time, Same Room, ONE LECTURE)
for k in 1:nr
    for j in 1:nt
        #think like k and j are constant. and use i value to check sum. change k&j and repeat
        @constraint(m,sum(TimeTables[i,j,k] for i in 1:nc) <= 1)
    end
end

#2.All courses takes two time slots
for i in 1:nc
    #each table has 2 ones.
    @constraint(m, sum(TimeTables[i,j,k] for j in 1:nt, k in 1:nr) == 2 )
end

#3. A course cannot be scheduled to two different places at the same time (no sections)(row sum 1 or 0)
for i in 1:nc
    for j in 1:nt
        #at first thinking like i and j are constant will be more understandable.
        @constraint(m, sum(TimeTables[i,j,k] for k in 1:nr) <= 1)
    end
end

#4. A professor cannot be in two different lectures at the same time
for p in 1:np #for prof1, prof2 ... profnp
    for j in 1:nt
        #prof p in jth hour can be in one room or zero 
        @constraint(m,sum(TimeTables[i,j,k] for i in findall(x->x==1, P[p,:]), k in 1:nr) <=1)
    end
end

#5. A student cannot be in two different lectures at the same time
for s in 1:ns #for student1, student2 ... studentns
    for j in 1:nt
        #student s in jth hour can be in one room or zero. 
        @constraint(m,sum(TimeTables[i,j,k] for i in findall(x->x==1, S[s,:]), k in 1:nr) <= 1)
    end
end

optimize!(m)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Oct  7 2019 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 0 - 0.00 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 72 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 72 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 72 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 63 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 41 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 40 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 39 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 20 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 15 strengthened rows, 0 substitutions
Cgl0004I processed model has 106 rows, 240 columns (240 integer (240 of which binary)) and 1218 elements
Cbc0045I No integer vari

In [4]:
print("\t|")
for k = 1:nr
    print("Room",k,"\t|")
end
for j = 1:nt
    println()
    print("Time",j,"\t|")
    for k = 1:nr
        print(sum(value(TimeTables[i,j,k])*i for i in 1:nc) ,"\t|")   
    end
end

	|Room1	|Room2	|Room3	|Room4	|
Time1	|8.0	|0.0	|10.0	|2.0	|
Time2	|0.0	|5.0	|4.0	|7.0	|
Time3	|1.0	|5.0	|10.0	|0.0	|
Time4	|2.0	|9.0	|3.0	|6.0	|
Time5	|7.0	|1.0	|0.0	|3.0	|
Time6	|8.0	|4.0	|9.0	|6.0	|